In [1]:
import tensorflow as tf
tf.config.gpu.set_per_process_memory_fraction(0.3)
tf.config.gpu.set_per_process_memory_growth(True)

In [2]:
import os
import numpy as np
import random
from random import shuffle
from skimage.io import imread,imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Activation, Dropout
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D
from tensorflow.keras.layers import Input, concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from itertools import chain

In [3]:
# Data Loader                                
def gen_list(path, directory):
    
    list_ = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if directory in r:
                list_.append(os.path.join(r, file))
    return list_

# Shuffle
def shuffle(list1, list2):
    
    combination = list(zip(list1, list2))
    random.shuffle(combination)
    list1, list2 = zip(*combination)
    
    return list1, list2

#Split for K-flod cross validation
def split_list(list_,k):
    list_store = []
    num = int(len(list_)/k)
    num_ = num+1
    remainder = len(list_)%k
    mid_k = k - remainder
    mid_ind = mid_k*num
    for j in range(k):
        if j < mid_k:
            list_store.append(list_[j*num : (j+1)*num])
        else:
            list_store.append(list_[mid_ind + (j-mid_k)*num_ : 
                                    mid_ind + (j-mid_k+1)*num_]) 
    
    return list_store


# Read data
def read_data(data_path, list_, img_h, img_w):
    data = []       
    for i in range(len(list_)):
        image_name = list_[i]
        img = imread(os.path.join(data_path, image_name), as_gray=True)
        img = resize(img, (img_h, img_w), anti_aliasing = True).astype('float32')
        #img = resize(img, (img_h, img_w), order=0, anti_aliasing=False, 
         #            preserve_range=True).astype('float32')

        data.append(np.array(img))
        
    data = np.expand_dims(data, axis = 3)    
    print('Reading finishied')         
    return data

def read_mask_onehot(data_path, list_, img_h, img_w):
    data = []       
    for i in range(len(list_)):
        image_name = list_[i]
        img = imread(os.path.join(data_path, image_name), as_gray=True)
        #img = resize(img, (img_h, img_w), anti_aliasing = True).astype('float32')
        img = resize(img, (img_h, img_w), order=0, anti_aliasing=False, 
                     preserve_range=True).astype('float32')

        data.append(np.array(img))
        
    # One-hot encoding
    M = []
    for m in np.unique(data):
        M.append(data==m)
    data = np.transpose(np.asarray(M), (1,2,3,0))    
    
    print('Reading finishied')         
    return data

'''def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + 
            smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)'''

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2 * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())


def precision(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + K.epsilon()) / (K.sum(y_pred_f) + K.epsilon())

def recall(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + K.epsilon()) / (K.sum(y_true_f) + K.epsilon())


def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def plot_learning_curve(History, fig_name):
    print('Epoch: '+ str(np.argmin(History.history["val_loss"])))
    print('Lowest Loss: '+ str(np.min(History.history["val_loss"])))
    %matplotlib inline
    plt.figure(figsize=(4, 4))
    plt.title("Learning curve")
    plt.plot(History.history["loss"], label="loss")
    plt.plot(History.history["val_loss"], label="val_loss")
    plt.plot( np.argmin(History.history["val_loss"]),
             np.min(History.history["val_loss"]),
             marker="x", color="r", label="best model")

    plt.xlabel("Epochs")
    plt.ylabel("Loss Value")
    plt.legend();
    plt.savefig('results/Lab4/' + fig_name + '.png')
    plt.close()
    
def plot_validation_metric(History, fig_name):
    print('Epoch: '+ str(np.argmax(History.history["val_dice_coef"])))
    print('Highest accuracy: '+ str(np.max(History.history["val_dice_coef"])))
    %matplotlib inline
    plt.figure(figsize=(4, 4))
    plt.title("Learning curve_metrics")
    plt.plot(History.history["val_dice_coef"], label="val_dice_coef")
    plt.plot(History.history["val_precision"], label="val_precision")
    plt.plot(History.history["val_recall"], label="val_recall")
    plt.plot( np.argmax(History.history["val_dice_coef"]),
             np.max(History.history["val_dice_coef"]),
             marker="x", color="r", label="best model")
    
    plt.xlabel("Epochs")
    plt.legend();
    plt.savefig('results/Lab4/' + fig_name + '.png')
    plt.close()

In [4]:
def conv_block_1(i, Base, acti, bn):
    n = Conv2D(Base, (3,3), padding='same')(i)
    n = BatchNormalization()(n) if bn else n
    n = Activation(acti)(n)
    n = Conv2D(Base, (3,3), padding='same')(n)
    n = BatchNormalization()(n) if bn else n
    o = Activation(acti)(n)
    return o

def conv_block_2(i, Base, acti, bn, drop):
    n = MaxPooling2D(pool_size=(2, 2))(i)
    n = Dropout(drop)(n) if drop else n
    o = conv_block_1(n, Base, acti, bn)
    return o

def conv_block_3(i, conca_i, Base, acti, bn, drop):
    n = Conv2DTranspose(Base, (2, 2), strides=(2, 2), padding='same')(i)
    n = concatenate([n, conca_i], axis=3)
    n = Dropout(drop)(n) if drop else n
    o = conv_block_1(n, Base, acti, bn)
    return o

def get_UNet(img_shape, Base, depth, inc_rate, activation, drop, batchnorm):
    i = Input(shape=img_shape)
    x_conca = []
    n = conv_block_1(i, Base, activation, batchnorm)
    x_conca.append(n)
    for k in range(depth):
        Base = Base*inc_rate
        n = conv_block_2(n, Base, activation, batchnorm, drop)
        if k < (depth-1):
            x_conca.append(n)
    for k in range(depth):
        Base = Base//inc_rate
        n = conv_block_3(n, x_conca[-1-k], Base, activation, batchnorm, drop)
        
    o = Conv2D(1, (1,1), activation='sigmoid')(n)
    
    return Model(inputs=i, outputs=o)

def get_UNet_multi(img_shape, Base, depth, inc_rate, activation, drop, batchnorm, N):
    i = Input(shape=img_shape)
    x_conca = []
    n = conv_block_1(i, Base, activation, batchnorm)
    x_conca.append(n)
    for k in range(depth):
        Base = Base*inc_rate
        n = conv_block_2(n, Base, activation, batchnorm, drop)
        if k < (depth-1):
            x_conca.append(n)
    for k in range(depth):
        Base = Base//inc_rate
        n = conv_block_3(n, x_conca[-1-k], Base, activation, batchnorm, drop)
        
    o = Conv2D(N, (1,1), activation='softmax')(n)
    
    return Model(inputs=i, outputs=o)

#model = get_UNet_multi(img_shape=(256,256,1), Base=16, depth=4, inc_rate=2, 
                 #activation='relu', drop=0.5, batchnorm=True)
#model.summary()

# Task 1

# k=3

In [5]:
path = '/Lab1/Lab3/MRI/'
img_h, img_w = 240, 240

k=3

Mask = gen_list(path, 'Mask')
Img = gen_list(path,'Image')
Mask, Img = shuffle(Mask, Img)

Mask = split_list(Mask,k)
Img = split_list(Img,k)

for i in range(k):
    Mask_val = list(Mask[i])
    Mask_train =list(chain.from_iterable(Mask[:i] + Mask[i+1:])) 
    Img_val = list(Img[i])
    Img_train =list(chain.from_iterable(Img[:i] + Img[i+1:])
    
    Mask_train = read_data(path+'Mask/', Mask_train, img_h, img_w)
    Mask_val =read_data(path+'Mask/', Mask_val, img_h, img_w)
    Img_train = read_data(path+'Image/', Img_train, img_h, img_w)
    Img_val = read_data(path+'Image/', Img_val, img_h, img_w)
    
    model = get_UNet(img_shape=(240,240,1), Base=16, depth=4, inc_rate=2, 
                 activation='relu', drop=0.5, batchnorm=True)

    model.compile(optimizer=Adam(lr=1e-5), loss=[dice_coef_loss], 
                  metrics=[dice_coef,precision,recall])

    History = model.fit(Img_train, Mask_train, batch_size=8, epochs=100, verbose=1, 
                        validation_data=(Img_val, Mask_val))

    plot_learning_curve(History, 'Task7_k={0}_loss_{1}'.format(k,i+1))
    plot_validation_metric(History, 'Task7_k={0}_metrics_{1}'.format(k,i+1))

Reading finishied
Reading finishied
Reading finishied
Reading finishied
Train on 6269 samples, validate on 3134 samples
Epoch 1/100
6269/6269 [==============================] - 133s 21ms/sample - loss: 0.2217 - dice_coef: 0.7783 - precision: 0.0202 - recall: 0.7918 - val_loss: 0.1523 - val_dice_coef: 0.8477 - val_precision: 0.0171 - val_recall: 0.5392
Epoch 2/100
6269/6269 [==============================] - 122s 19ms/sample - loss: 0.1469 - dice_coef: 0.8531 - precision: 0.0178 - recall: 0.5348 - val_loss: 0.1271 - val_dice_coef: 0.8729 - val_precision: 0.0161 - val_recall: 0.4534
Epoch 3/100
6269/6269 [==============================] - 123s 20ms/sample - loss: 0.1189 - dice_coef: 0.8811 - precision: 0.0161 - recall: 0.4243 - val_loss: 0.1159 - val_dice_coef: 0.8841 - val_precision: 0.0159 - val_recall: 0.4222
Epoch 4/100
6269/6269 [==============================] - 122s 19ms/sample - loss: 0.1083 - dice_coef: 0.8917 - precision: 0.0156 - recall: 0.3908 - val_loss: 0.1114 - val_dice_co

Epoch 70/100
6269/6269 [==============================] - 119s 19ms/sample - loss: 0.0071 - dice_coef: 0.9929 - precision: 0.0264 - recall: 0.0477 - val_loss: 0.0073 - val_dice_coef: 0.9927 - val_precision: 0.0274 - val_recall: 0.0523
Epoch 71/100
6269/6269 [==============================] - 118s 19ms/sample - loss: 0.0071 - dice_coef: 0.9929 - precision: 0.0267 - recall: 0.0458 - val_loss: 0.0074 - val_dice_coef: 0.9926 - val_precision: 0.0288 - val_recall: 0.0529
Epoch 72/100
6269/6269 [==============================] - 118s 19ms/sample - loss: 0.0070 - dice_coef: 0.9930 - precision: 0.0270 - recall: 0.0442 - val_loss: 0.0069 - val_dice_coef: 0.9931 - val_precision: 0.0273 - val_recall: 0.0458
Epoch 73/100
6269/6269 [==============================] - 117s 19ms/sample - loss: 0.0070 - dice_coef: 0.9930 - precision: 0.0273 - recall: 0.0426 - val_loss: 0.0070 - val_dice_coef: 0.9930 - val_precision: 0.0290 - val_recall: 0.0461
Epoch 74/100
6269/6269 [==============================] - 11

6269/6269 [==============================] - 118s 19ms/sample - loss: 0.0468 - dice_coef: 0.9532 - precision: 0.0151 - recall: 0.2326 - val_loss: 0.0403 - val_dice_coef: 0.9597 - val_precision: 0.0177 - val_recall: 0.2377
Epoch 5/100
6269/6269 [==============================] - 119s 19ms/sample - loss: 0.0393 - dice_coef: 0.9607 - precision: 0.0150 - recall: 0.2088 - val_loss: 0.0368 - val_dice_coef: 0.9632 - val_precision: 0.0177 - val_recall: 0.2219
Epoch 6/100
6269/6269 [==============================] - 118s 19ms/sample - loss: 0.0344 - dice_coef: 0.9656 - precision: 0.0150 - recall: 0.1910 - val_loss: 0.0329 - val_dice_coef: 0.9671 - val_precision: 0.0174 - val_recall: 0.2066
Epoch 7/100
6269/6269 [==============================] - 118s 19ms/sample - loss: 0.0304 - dice_coef: 0.9696 - precision: 0.0149 - recall: 0.1769 - val_loss: 0.0308 - val_dice_coef: 0.9692 - val_precision: 0.0176 - val_recall: 0.1972
Epoch 8/100
6269/6269 [==============================] - 118s 19ms/sample - 

Epoch 39/100
6269/6269 [==============================] - 118s 19ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0236 - recall: 0.0293 - val_loss: 0.0077 - val_dice_coef: 0.9923 - val_precision: 0.0282 - val_recall: 0.0368
Epoch 40/100
6269/6269 [==============================] - 118s 19ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0246 - recall: 0.0287 - val_loss: 0.0076 - val_dice_coef: 0.9924 - val_precision: 0.0293 - val_recall: 0.0365
Epoch 41/100
6269/6269 [==============================] - 118s 19ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0255 - recall: 0.0279 - val_loss: 0.0069 - val_dice_coef: 0.9931 - val_precision: 0.0278 - val_recall: 0.0276
Epoch 42/100
6269/6269 [==============================] - 118s 19ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0266 - recall: 0.0275 - val_loss: 0.0069 - val_dice_coef: 0.9931 - val_precision: 0.0293 - val_recall: 0.0287
Epoch 43/100
6269/6269 [==============================] - 11

Epoch 74/100
6269/6269 [==============================] - 81s 13ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0540 - recall: 0.0314 - val_loss: 0.0069 - val_dice_coef: 0.9931 - val_precision: 0.0480 - val_recall: 0.0182
Epoch 75/100
6269/6269 [==============================] - 80s 13ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0543 - recall: 0.0312 - val_loss: 0.0069 - val_dice_coef: 0.9931 - val_precision: 0.0473 - val_recall: 0.0155
Epoch 76/100
6269/6269 [==============================] - 80s 13ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0544 - recall: 0.0313 - val_loss: 0.0069 - val_dice_coef: 0.9931 - val_precision: 0.0454 - val_recall: 0.0132
Epoch 77/100
6269/6269 [==============================] - 80s 13ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0550 - recall: 0.0319 - val_loss: 0.0069 - val_dice_coef: 0.9931 - val_precision: 0.0473 - val_recall: 0.0151
Epoch 78/100
6269/6269 [==============================] - 81s 13

6268/6268 [==============================] - 80s 13ms/sample - loss: 0.0071 - dice_coef: 0.9929 - precision: 0.0261 - recall: 0.0408 - val_loss: 0.0113 - val_dice_coef: 0.9887 - val_precision: 0.0339 - val_recall: 0.0796
Epoch 44/100
6268/6268 [==============================] - 81s 13ms/sample - loss: 0.0071 - dice_coef: 0.9929 - precision: 0.0263 - recall: 0.0390 - val_loss: 0.0114 - val_dice_coef: 0.9886 - val_precision: 0.0340 - val_recall: 0.0796
Epoch 45/100
6268/6268 [==============================] - 81s 13ms/sample - loss: 0.0070 - dice_coef: 0.9930 - precision: 0.0265 - recall: 0.0367 - val_loss: 0.0107 - val_dice_coef: 0.9893 - val_precision: 0.0343 - val_recall: 0.0724
Epoch 46/100
6268/6268 [==============================] - 81s 13ms/sample - loss: 0.0070 - dice_coef: 0.9930 - precision: 0.0271 - recall: 0.0358 - val_loss: 0.0098 - val_dice_coef: 0.9902 - val_precision: 0.0338 - val_recall: 0.0654
Epoch 47/100
6268/6268 [==============================] - 81s 13ms/sample - l

# k=5

In [5]:
path = '/Lab1/Lab3/MRI/'
img_h, img_w = 240, 240

k=5

Mask = gen_list(path, 'Mask')
Img = gen_list(path,'Image')
Mask, Img = shuffle(Mask, Img)

Mask = split_list(Mask,k)
Img = split_list(Img,k)

for i in range(k):
    Mask_val = list(Mask[i])
    Mask_train =list(chain.from_iterable(Mask[:i] + Mask[i+1:])) 
    Img_val = list(Img[i])
    Img_train =list(chain.from_iterable(Img[:i] + Img[i+1:]))
    
    
    Mask_train = read_data(path+'Mask/', Mask_train, img_h, img_w)
    Mask_val =read_data(path+'Mask/', Mask_val, img_h, img_w)
    Img_train = read_data(path+'Image/', Img_train, img_h, img_w)
    Img_val = read_data(path+'Image/', Img_val, img_h, img_w)
    
    model = get_UNet(img_shape=(240,240,1), Base=16, depth=4, inc_rate=2, 
                 activation='relu', drop=0.5, batchnorm=True)

    model.compile(optimizer=Adam(lr=1e-5), loss=[dice_coef_loss], 
                  metrics=[dice_coef,precision,recall])

    History = model.fit(Img_train, Mask_train, batch_size=8, epochs=100, verbose=1, 
                        validation_data=(Img_val, Mask_val))

    plot_learning_curve(History, 'Task7_k={0}_loss_{1}'.format(k,i+1))
    plot_validation_metric(History, 'Task7_k={0}_metrics_{1}'.format(k,i+1))

Reading finishied
Reading finishied
Reading finishied
Reading finishied
Train on 7523 samples, validate on 1880 samples
Epoch 1/100
7523/7523 [==============================] - 103s 14ms/sample - loss: 0.1849 - dice_coef: 0.8151 - precision: 0.0211 - recall: 0.7281 - val_loss: 0.1197 - val_dice_coef: 0.8803 - val_precision: 0.0200 - val_recall: 0.5232
Epoch 2/100
7523/7523 [==============================] - 95s 13ms/sample - loss: 0.1225 - dice_coef: 0.8775 - precision: 0.0217 - recall: 0.5797 - val_loss: 0.0881 - val_dice_coef: 0.9119 - val_precision: 0.0206 - val_recall: 0.4480
Epoch 3/100
7523/7523 [==============================] - 95s 13ms/sample - loss: 0.0889 - dice_coef: 0.9111 - precision: 0.0220 - recall: 0.4832 - val_loss: 0.0748 - val_dice_coef: 0.9252 - val_precision: 0.0214 - val_recall: 0.4216
Epoch 4/100
7523/7523 [==============================] - 95s 13ms/sample - loss: 0.0759 - dice_coef: 0.9241 - precision: 0.0222 - recall: 0.4415 - val_loss: 0.0698 - val_dice_coef:

7523/7523 [==============================] - 53s 7ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0433 - recall: 0.0257 - val_loss: 0.0067 - val_dice_coef: 0.9933 - val_precision: 0.0445 - val_recall: 0.0261
Epoch 71/100
7523/7523 [==============================] - 53s 7ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0441 - recall: 0.0261 - val_loss: 0.0067 - val_dice_coef: 0.9933 - val_precision: 0.0453 - val_recall: 0.0267
Epoch 72/100
7523/7523 [==============================] - 53s 7ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0447 - recall: 0.0262 - val_loss: 0.0067 - val_dice_coef: 0.9933 - val_precision: 0.0453 - val_recall: 0.0264
Epoch 73/100
7523/7523 [==============================] - 533s 71ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0457 - recall: 0.0268 - val_loss: 0.0067 - val_dice_coef: 0.9933 - val_precision: 0.0454 - val_recall: 0.0259
Epoch 74/100
7523/7523 [==============================] - 52s 7ms/sample - loss

7523/7523 [==============================] - 54s 7ms/sample - loss: 0.0070 - dice_coef: 0.9930 - precision: 0.0239 - recall: 0.0327 - val_loss: 0.0118 - val_dice_coef: 0.9882 - val_precision: 0.0331 - val_recall: 0.0806
Epoch 40/100
7523/7523 [==============================] - 54s 7ms/sample - loss: 0.0070 - dice_coef: 0.9930 - precision: 0.0248 - recall: 0.0318 - val_loss: 0.0448 - val_dice_coef: 0.9552 - val_precision: 0.0333 - val_recall: 0.2572
Epoch 41/100
7523/7523 [==============================] - 55s 7ms/sample - loss: 0.0070 - dice_coef: 0.9930 - precision: 0.0256 - recall: 0.0310 - val_loss: 0.0121 - val_dice_coef: 0.9879 - val_precision: 0.0350 - val_recall: 0.0805
Epoch 42/100
7523/7523 [==============================] - 55s 7ms/sample - loss: 0.0069 - dice_coef: 0.9931 - precision: 0.0264 - recall: 0.0302 - val_loss: 0.0126 - val_dice_coef: 0.9874 - val_precision: 0.0349 - val_recall: 0.0811
Epoch 43/100
7523/7523 [==============================] - 54s 7ms/sample - loss: 

7522/7522 [==============================] - 54s 7ms/sample - loss: 0.0300 - dice_coef: 0.9700 - precision: 0.0160 - recall: 0.1884 - val_loss: 0.0299 - val_dice_coef: 0.9701 - val_precision: 0.0178 - val_recall: 0.2026
Epoch 9/100
7522/7522 [==============================] - 54s 7ms/sample - loss: 0.0267 - dice_coef: 0.9733 - precision: 0.0160 - recall: 0.1748 - val_loss: 0.0267 - val_dice_coef: 0.9733 - val_precision: 0.0176 - val_recall: 0.1869
Epoch 10/100
7522/7522 [==============================] - 54s 7ms/sample - loss: 0.0239 - dice_coef: 0.9761 - precision: 0.0161 - recall: 0.1625 - val_loss: 0.0242 - val_dice_coef: 0.9758 - val_precision: 0.0178 - val_recall: 0.1766
Epoch 11/100
7522/7522 [==============================] - 54s 7ms/sample - loss: 0.0213 - dice_coef: 0.9787 - precision: 0.0162 - recall: 0.1509 - val_loss: 0.0224 - val_dice_coef: 0.9776 - val_precision: 0.0183 - val_recall: 0.1701
Epoch 12/100
7522/7522 [==============================] - 54s 7ms/sample - loss: 0

7522/7522 [==============================] - 54s 7ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0614 - recall: 0.0338 - val_loss: 0.0067 - val_dice_coef: 0.9933 - val_precision: 0.0632 - val_recall: 0.0343
Epoch 79/100
7522/7522 [==============================] - 54s 7ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0615 - recall: 0.0339 - val_loss: 0.0067 - val_dice_coef: 0.9933 - val_precision: 0.0635 - val_recall: 0.0317
Epoch 80/100
7522/7522 [==============================] - 54s 7ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0619 - recall: 0.0340 - val_loss: 0.0067 - val_dice_coef: 0.9933 - val_precision: 0.0634 - val_recall: 0.0315
Epoch 81/100
7522/7522 [==============================] - 54s 7ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0621 - recall: 0.0339 - val_loss: 0.0067 - val_dice_coef: 0.9933 - val_precision: 0.0639 - val_recall: 0.0319
Epoch 82/100
7522/7522 [==============================] - 54s 7ms/sample - loss: 

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7522/7522 [==============================] - 53s 7ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0620 - recall: 0.0337 - val_loss: 0.0066 - val_dice_coef: 0.9934 - val_precision: 0.0577 - val_recall: 0.0182
Epoch 91/100
7522/7522 [==============================] - 53s 7ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0621 - recall: 0.0341 - val_loss: 0.0066 - val_dice_coef: 0.9934 - val_precision: 0.0593 - val_recall: 0.0204
Epoch 92/100
7522/7522 [==============================] - 53s 7ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0621 - recall: 0.0339 - val_loss: 0.0066 - val_dice_coef: 0.9934 - val_precision: 0.0599 - val_recall: 0.0201
Epoch 93/100
4936/7522 [==================>...........] - ETA: 16s - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0634 - recall: 0.0347

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7522/7522 [==============================] - 53s 7ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0631 - recall: 0.0341 - val_loss: 0.0066 - val_dice_coef: 0.9934 - val_precision: 0.0617 - val_recall: 0.0222
Epoch 100/100
7522/7522 [==============================] - 54s 7ms/sample - loss: 0.0067 - dice_coef: 0.9933 - precision: 0.0634 - recall: 0.0345 - val_loss: 0.0066 - val_dice_coef: 0.9934 - val_precision: 0.0623 - val_recall: 0.0219
Epoch: 63
Lowest Loss: 0.0065491649503921145
Epoch: 63
Highest accuracy: 0.9934506
Reading finishied
Reading finishied
Reading finishied
Reading finishied
Train on 7522 samples, validate on 1881 samples
Epoch 1/100
7522/7522 [==============================] - 1107s 147ms/sample - loss: 0.1299 - dice_coef: 0.8701 - precision: 0.0151 - recall: 0.4180 - val_loss: 0.0805 - val_dice_coef: 0.9195 - val_precision: 0.0176 - val_recall: 0.3452
Epoch 2/100
7522/7522 [==============================] - 53s 7ms/sample - loss: 0.0733 - dice_coef: 0.9267 

7522/7522 [==============================] - 58s 8ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0565 - recall: 0.0323 - val_loss: 0.0070 - val_dice_coef: 0.9930 - val_precision: 0.0612 - val_recall: 0.0344
Epoch 69/100
7522/7522 [==============================] - 57s 8ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0565 - recall: 0.0321 - val_loss: 0.0070 - val_dice_coef: 0.9930 - val_precision: 0.0609 - val_recall: 0.0361
Epoch 70/100
7522/7522 [==============================] - 58s 8ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0567 - recall: 0.0323 - val_loss: 0.0070 - val_dice_coef: 0.9930 - val_precision: 0.0601 - val_recall: 0.0349
Epoch 71/100
7522/7522 [==============================] - 57s 8ms/sample - loss: 0.0066 - dice_coef: 0.9934 - precision: 0.0565 - recall: 0.0322 - val_loss: 0.0070 - val_dice_coef: 0.9930 - val_precision: 0.0610 - val_recall: 0.0343
Epoch 72/100
7522/7522 [==============================] - 57s 8ms/sample - loss: 